In [8]:
import pickle
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
pkl_file = open('df.pkl', 'rb')
df = pickle.load(pkl_file)
pkl_file.close() 

In [3]:
df.shape

(1125, 16)

In [4]:
df.head()

,trail_name,top_elev_(ft),bottom_elev_(ft),vert_rise_(ft),slope_length_(ft),avg_width_(ft),slope_area_(acres),avg_grade_(%),max_grade_(%),ability_level,resort,location,groomed,colors,ability_nums,color_nums
0,Over the Rainbow,11924.0,10858.0,1066.0,2404.0,319,17.6,50.0,69.0,Expert,Loveland,CO,0,bb,6,4
1,Zoom,11405.0,11005.0,400.0,839.0,195,3.7,55.0,66.0,Expert,Loveland,CO,0,bb,6,4
2,Avalanche Bowl,11840.0,11405.0,435.0,1141.0,255,6.7,42.0,76.0,Expert,Loveland,CO,0,bb,6,4
3,Tiger's Tail,11856.0,11469.0,386.0,1183.0,136,3.7,35.0,61.0,Expert,Loveland,CO,0,black,6,3
4,Spillway,11849.0,11631.0,218.0,702.0,198,3.2,33.0,41.0,Intermediate,Loveland,CO,1,blue,4,2


In [34]:
df = df.reset_index(drop=True)

In [35]:
features = ['top_elev_(ft)', 
            'bottom_elev_(ft)', 
            'vert_rise_(ft)', 
            'slope_length_(ft)', 
            'avg_width_(ft)', 
            'slope_area_(acres)', 
            'avg_grade_(%)', 
            'max_grade_(%)', 
            'groomed']

X = df[features].values

In [36]:
X

array([[11924.0, 10858.0, 1066.0, ..., 50.0, 69.0, 0],
       [11405.0, 11005.0, 400.0, ..., 55.0, 66.0, 0],
       [11840.0, 11405.0, 435.0, ..., 42.0, 76.0, 0],
       ..., 
       [8110.0, 8100.0, 10.0, ..., 8.0, 11.0, 1],
       [8113.0, 8100.0, 13.0, ..., 12.0, 12.0, 1],
       [8147.0, 8100.0, 47.0, ..., 15.0, 12.0, 1]], dtype=object)

In [133]:
def cos_sim_recommendations_resort(trail_name, resort_name, X, n=5, resort=None):
    index = df.index[(df['trail_name'] == trail_name) & (df['resort'] == resort_name)][0]
    trail = X[index].reshape(1,-1)
    cs = cosine_similarity(trail, X)
    rec_index = np.argsort(cs)[0][::-1][1:]
    ordered_df = df.loc[rec_index]
    if resort:
        ordered_df = ordered_df[ordered_df['resort'] == resort]
    rec_df = ordered_df.head(n)
    orig_row = df.loc[[index]].rename(lambda x: 'original')
    total = pd.concat((orig_row,rec_df))
    return total

In [135]:
cos_sim_recommendations_resort('Hook Up','Winter Park',X,n=5,resort= None)

,trail_name,top_elev_(ft),bottom_elev_(ft),vert_rise_(ft),slope_length_(ft),avg_width_(ft),slope_area_(acres),avg_grade_(%),max_grade_(%),ability_level,resort,location,groomed,colors,ability_nums,color_nums
original,Hook Up,10670.0,10530.0,140.0,721.44,106,2.1,20.0,20.0,Novice,Winter Park,CO,0,green,2,1
707,Augusta,9414.0,9311.0,103.0,630.00,130,1.9,17.0,23.0,Novice,Crested Butte,CO,1,green,2,1
682,Big Al’s,9854.0,9747.0,107.0,712.00,93,1.5,15.0,28.0,Novice,Crested Butte,CO,1,green,2,1
720,Paradise Access,9984.0,9833.0,151.0,723.00,81,1.4,21.0,33.0,Low Intermediate,Crested Butte,CO,0,blue,3,2
1028,Upper Sheephorn,11153.0,11014.0,138.0,793.00,175,3.2,18.0,32.0,Low Intermediate,Beaver Creek,CO,1,green,3,1
817,Rueggli,9413.0,9341.0,72.0,605.00,98,1.4,12.0,16.0,Novice,Taos,NM,1,green,2,1
